In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

In [36]:
dss.run_command('Redirect 05node_singlephase_balanced_oscillation_03.dss') #load dss file
dss.Solution.Solve() #solve

Asides: 
 - what does it look like when the r and x matrix have all the parts? 
 - are the negatives on the kvl c and d properly placed?
 

### XNR

In [14]:
nnode = len(dss.Circuit.AllBusNames()) 
nline = len(dss.Lines.AllNames())

#this is old code, but it makes XNR

V_all = np.array([])
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    v_temp = dss.Bus.PuVoltage()
    z = np.zeros(6) #preallocate for re/im in 3 phases
    z[:len(v_temp)] = np.array([v_temp])   
    V_all = np.append(V_all, z)
V_all = np.reshape(V_all, (nnode, 6)).T
b_SB = V_all[:,:1]


I_all = np.array([])
for k2 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k2]) 
    i_temp = np.divide(dss.CktElement.Currents(), dss.Bus.kVBase())
    z = np.zeros(6)
    z[:len(i_temp)] = i_temp 
    I_all = np.append(I_all, z)
I_all = np.reshape(I_all, (nline, 6)).T

XNR = np.zeros((2*3*(nnode+nline),1))

for ph in range(0,3):
    for k1 in range(0,nnode):
        XNR[2*ph*nnode + 2*k1] = V_all[ph * 2 , k1]
        XNR[2*ph*nnode + 2*k1+1] = V_all[(ph * 2) +1,k1]

for ph in range(0,3):
    for k1 in range(0,nline):
        XNR[(2*3*nnode) + 2*ph*nline + 2*k1] = I_all[ph * 2,k1]
        XNR[(2*3*nnode) + 2*ph*nline + (2*k1)+1] = I_all[(ph * 2) + 1,k1]


[[  9.99996847e-01   9.78981009e-01   9.78036549e-01   9.76619109e-01
    9.76146616e-01   9.76146384e-01   9.75673640e-01]
 [ -5.13569112e-06  -3.60756228e-02  -3.63733986e-02  -3.68200381e-02
   -3.69688969e-02  -3.69689191e-02  -3.71177683e-02]
 [ -4.99999967e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -8.66025072e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -4.99999968e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  8.66025735e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]]
(6, 7)
[[  9.99996847e-01]
 [ -5.13569112e-06]
 [ -4.99999967e-01]
 [ -8.66025072e-01]
 [ -4.99999968e-01]
 [  8.66025735e-01]]
[ 52.75281871 -15.31629705 -52.75278047  15.31732994   0.           0.        ]
[ 39.58154305 -11.49917995 -39.58150442  11.500

### Slack Bus

$$ \delta_{SBRE,m} = A_{m}^{\phi} - \Re \left\{ V_{sl}^{\phi} \right\} \\
\delta_{SBIM,m} = B_{m}^{\phi} - \Im \left\{ V_{sl}^{\phi} \right\} $$
 --- 
 
 $$ {\Delta}_{SB} = {G}_{SB} {X} + {b}_{SB} $$
 ---
$$ X $$

In [29]:
line_names = dss.Lines.AllNames()

A_m = np.array([])
B_m = np.array([])

C_mn = np.array([])
D_mn = np.array([])

R_matrix = np.zeros((nline,9))
X_matrix = np.zeros((nline,9))

dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
kV_base = dss.Bus.kVBase()


for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    
    volts = dss.Bus.PuVoltage() #get bus1's puVoltage
    a_temp = np.zeros(3)
    b_temp = np.zeros(3)
    
    for i in range(0, len(volts), 2):
        a_temp[i//2] = volts[i]
        b_temp[i//2] = volts[i+1]
    A_m = np.append(A_m, a_temp) #split into re/im parts
    B_m = np.append(B_m, b_temp)

for k2 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k2]) #set the line
   
    linecode = dss.Lines.LineCode() #get the linecode
    dss.LineCodes.Name(linecode) #set the linecode
    
    xmat = dss.LineCodes.Xmatrix() #get the xmat 
    rmat = dss.LineCodes.Rmatrix() #get the rmat
    
    for i in range(len(xmat)):
        X_matrix[k2][i] = xmat[i] #fill x/r where they are shaped like nline x 9 (for 9 components)
    for j in range(len(rmat)):
        R_matrix[k2][i] = rmat[j]
 
    c_temp = np.zeros(3) #retrieve line current
    d_temp = np.zeros(3)
    
    for i in range(0, len(dss.CktElement.Currents()), 2): #get the currents of the line
        c_temp[i//2] = np.divide(dss.CktElement.Currents()[i], kV_base) #per unit-ify the currents
        d_temp[i//2] = np.divide(dss.CktElement.Currents()[i+1], kV_base) 
    C_mn = np.append(C_mn, c_temp)
    D_mn = np.append(D_mn, d_temp)
    
X = np.array([]) #make X, should be 2*3*(nline+nnode) long

for ph in range(0,3):
    for nodes in range(nnode):
        X = np.append(X, A_m[nodes*3 + ph]) #add a, b by node and then phase
        X = np.append(X, B_m[nodes*3 + ph])

for ph in range(0, 3):
    for lines in range(nline):
        X = np.append(X, C_mn[lines*3 + ph]) #add c, d by line and then phase
        X = np.append(X, D_mn[lines*3 + ph])

print(X.shape)

(72,)


$$ G_{SB} $$ 

In [23]:
g_SB = np.array([]) #assumes slack bus is at index 0 
sb_idx = [0, 1, 2*nnode, 2*nnode+1, 3*nnode, 3*nnode+1]
for i in range(len(sb_idx)):    
    temp_row = np.zeros(len(X))
    temp_row[sb_idx[i]] = 1
    g_SB = np.append(g_SB, temp_row)
g_SB = np.reshape(g_SB, (6, len(g_SB) // 6))
## print(g_SB)
print(g_SB.shape)


(6, 72)


$$ b_{SB} $$ 

In [32]:
b_SB = np.array([])
sb_idx = [0, 1, 2*nnode, 2*nnode+1, 3*nnode, 3*nnode+1]
for i in range(len(sb_idx)):
    b_SB = np.append(b_SB, X[sb_idx[i]])
print(b_SB)
print(b_SB.shape)


[  9.99996847e-01  -5.13569112e-06  -4.99999967e-01  -8.66025072e-01
   0.00000000e+00   0.00000000e+00]
(6,)


### KVL

Need an example with a resistance reactance matrix (larger than just 1 element), and need to put it in G_KVL correctly (adapt to three phases)

X is like $$[a^1_a, b^1_a, a^2_a, b^1_a, ...a^{nnode}_a, b^{nnode}_a, a^1_b, b^1_b, .... a^{nnode}_b, a^{nnode}_b, a1_c, b1_c, ... a^{nnode}_c, b^{nnode}_c, c_{mn, a}^1, d_{mn, a}^1, ..., c_{mn,a}^{nlinelast}, d_{mn, a}^{nlinelast}, c_{mn, b}^1, d_{mn, b}^1, ... , c_{mn, b}^{nlinelast}, d_{mn, b}^{nlinelast}, c_{mn,c}^{1}, d_{mn, c}^{1}, ..., c_{mn,c}^{nlinelast}, d_{mn, c}^{nlinelast} ] $$

---

Helper Method

In [63]:
def get_bus_idx(bus):
    k = -1
    for n in range(len(dss.Circuit.AllBusNames())):
        if dss.Circuit.AllBusNames()[n] in bus:
            k = n
    return k
dss.Lines.Name(dss.Lines.AllNames()[0]) #set the line
bus1 = dss.Lines.Bus1()
get_bus_idx(bus1)


1

$$ G_{KVL} $$ 

In [73]:
G_KVL = np.array([])
first_template = np.array([1, 0, -1, 0]) #first eqn coeff
second_template = np.array([0, 1, 0, -1]) #second eqn coeff

for ph in range(0, 3):
    for line in range(len(dss.Lines.AllNames())):    
        dss.Lines.Name(dss.Lines.AllNames()[line]) #set the line
        bus1 = dss.Lines.Bus1()
        bus2 = dss.Lines.Bus2()
        
        bus1_idx = get_bus_idx(bus1)
        bus2_idx = get_bus_idx(bus2)

        temp_row = np.zeros(len(X))
        temp_row[2*nnode*ph + 2*bus1_idx] = 1 #A_m
        temp_row[2*nnode*ph + 2*bus2_idx] = -1 #A_n
        G_KVL = np.append(G_KVL, temp_row)
        
        temp_row = np.zeros(len(X))
        temp_row[2*nnode*ph + 2*bus1_idx + 1] = -1 #B_m
        temp_row[2*nnode*ph + 2*bus2_idx + 1] = 1 #B_n
        G_KVL = np.append(G_KVL, temp_row)

G_KVL = np.reshape(G_KVL,(2*3*nline, len(X)) )

print(G_KVL.shape) #(6*line)x(2*3*(nline+nnode)) bc
#each line is complex (2) and has 3 phases
for ph in range(0,3):
    for line in range(0,len(dss.Lines.AllNames())):
        r_temp = sum(R_matrix[line, :]) #add up all the line resistance comp 
        x_temp = sum(X_matrix[line, :]) #add up all the line react comp
        G_KVL[2 * ph * line][2*3*nnode + 2*line*ph] = -r_temp #C_mn #set g_kvl as such
        G_KVL[2 * ph * line][2*3*nnode + 2*line*ph + 1] = -x_temp #D_mn
        G_KVL[2 * ph * line + 1][2*3*nnode + 2*line*ph] = -x_temp #C_mn
        G_KVL[2 * ph * line + 1][2*3*nnode + 2*line*ph + 1] = -r_temp #D_mn
        line_counter += 1

# import sys
# np.set_printoptions(threshold=sys.maxsize)
# print(G_KVL)


(30, 72)


$$ b_{KVL} $$

In [71]:
b_kvl = np.zeros(len(G_KVL))

### KCL

How do you get capacitance

Helpful Methods

In [74]:
def linelist(busname): 
    in_lines = np.array([])
    out_lines = np.array([])
    for k in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k])
        if busname in dss.Lines.Bus1():
            out_lines = np.append(out_lines, dss.Lines.AllNames()[k])
        elif busname in dss.Lines.Bus2():
            in_lines = np.append(in_lines,dss.Lines.AllNames()[k])
    return in_lines,out_lines
        
il , ol = linelist(dss.Circuit.AllBusNames()[3])


def get_line_idx(line):
    k = -1
    for n in range(len(dss.Lines.AllNames())):
        if dss.Lines.AllNames()[n] == line:
            k = n
    return k


In [75]:
beta_S = 0.85 
beta_I = 0.05
beta_Z = 0.1 

In [76]:
H = np.zeros((2 * 3 * (nnode + nline), 2 * 3* (nnode + nline), 3)) 
#(2*3*(nnode+nline)) x (2*3*(nnode+nline)) x (2*3*nline) 
    #(for each phase, real/im, line)
g = np.zeros((len(X), 3)) #1 x (2*3*(nnode+nline))
b = np.zeros(3) #scalar I guess

def get_line_idx(line):
    k = -1
    for n in range(len(dss.Lines.AllNames())):
        if dss.Lines.AllNames()[n] == line:
            k = n
    return k

def KCL_residuals(cplx):
    for ph in range(0,3):
        if ph == 0:
            A0 = 1
            B0 = 0
        elif ph == 1:
            A0 = -1/2
            B0 = -1 * np.sqrt(3)/2
        elif ph == 2:
            A0 = -1/2
            B0 = np.sqrt(3)/2
        for k2 in range(len(dss.Circuit.AllBusNames())):
            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])     
            #dss.Lines.Name(dss.Lines.AllNames()[k1])
            in_lines, out_lines = linelist(dss.Circuit.AllBusNames()[k2])

#             dA = XNR[idxAm] - A0
#             dB = XNR[idxBm] - B0

#             dX = np.array([dA[0], dB[0]]) #(1, 2)
#             dX_t = np.array([dA[0], dB[0]]).T #(2,1)

            gradient_mag = np.array([A0 * ((A0**2+B0**2) ** (-1/2)), B0 * ((A0**2+B0**2) ** (-1/2))])
            hessian_mag = np.array([[-((A0**2)*(A0**2+B0**2)**(-3/2))+(A0**2+B0**2)**(-1/2), -A0*B0*(A0**2+B0**2)**(-3/2)],
                                [-A0*B0*(A0**2+B0**2)**(-3/2), -((B0**2)*(A0**2+B0**2)**(-3/2))+((A0**2+B0**2)**(-1/2))]])
#             first_order_term_mag =  np.matmul(gradient_mag, dX_t)
#             second_order_term_mag = (1/2) * np.matmul(dX_t, np.matmul(hessian_mag, dX))

            H[2*nnode*ph + 2*k2,2*nnode*ph + 2*k2,ph] = beta_Z + (0.5 * beta_I * hessian_mag[0][0])
            H[2*nnode*ph + 2*k2 + 1][2*nnode*ph + 2*k2 +1][ph] = beta_Z + (0.5 * beta_I * hessian_mag[1][1])
            H[2*nnode*ph + 2*k2][2*nnode*ph + 2*k2 + 1][ph] = 0.5 + (beta_I * hessian_mag[0][1])
            H[2*nnode*ph + 2*k2 + 1][2*nnode*ph + 2*k2][ph] = 0.5 + (beta_I * hessian_mag[0][1])


            for i in range(len(in_lines)): #fill in H for the inlines
                dss.Lines.Name(in_lines[i])
                line_idx = get_line_idx(in_lines[i])
                if cplx == 're': #real residual
                    #A_m and C_lm
                    H[2*nnode*ph + 2*k2][2*3*nnode + 2*ph*line_idx][ph] = 1/2
                    H[2*3*nnode + 2*ph*line_idx][2*nnode*ph + 2*k2][ph] = 1/2
                    #B_m and D_lm
                    H[2*nnode*ph+2*k2+1][2*3*nnode+2*ph*line_idx+1][ph] = 1/2
                    H[2*3*nnode+2*ph*line_idx+1][2*nnode*ph+2*k2+1][ph] = 1/2
                if cplx == 'im': #complex residual
                    #AD
                    H[2*nnode*ph + 2*k2][2*3*nnode + 2 *ph * line_idx + 1][ph] = -1/2
                    H[2*3*nnode + 2 *ph * line_idx + 1][2*nnode*ph + 2*k2][ph] = -1/2
                    #B_m and C_lm
                    H[2*nnode*ph + 2*k2 + 1][2*3*nnode + 2 *ph * line_idx][ph] = 1/2
                    H[2*3*nnode + 2 * ph *line_idx][2*nnode*ph + 2*k2 + 1][ph] = 1/2
                    
            for j in range(len(out_lines)): #fill in H for the outlines
                dss.Lines.Name(out_lines[j])
                line_idx = get_line_idx(out_lines[j])
                k = get_bus_idx(dss.Lines.Bus2())
                if cplx == 're':
                    #A_m and C_mn
                    H[2*nnode*ph + 2*k2][2*3*nnode + 2 * ph *line_idx][ph] = -1/2
                    H[2*3*nnode + 2 *ph * line_idx][2*nnode*ph + 2*k2][ph] = -1/2
                    #B_m and D_mn
                    H[2*nnode*ph + 2*k2 + 1][2*3*nnode + 2 *ph * line_idx + 1][ph] = -1/2
                    H[2*3*nnode + 2 *ph * line_idx + 1][2*nnode*ph + 2*k2 + 1][ph] = -1/2
                if cplx == 'im':
                    #A_m and D_mn
                    H[2*nnode*ph + 2*k2][2*3*nnode + 2 *ph * line_idx + 1][ph] = 1/2
                    H[2*3*nnode + 2 * ph * line_idx + 1][2 * nnode * ph + 2 * k2][ph] = 1/2
                    #C_m and B_mn
                    H[2*nnode*ph + 2*k2 + 1][2*3*nnode + 2 * ph * line_idx][ph] = -1/2
                    H[2*3*nnode + 2 * ph * line_idx ][2*nnode*ph + 2 * k2 + 1][ph] = -1/2

        #inside ph loop
        g_temp = np.zeros(len(X))
        for i in range(0, len(g), 4):
            g_temp[i] = -1/2 * beta_I * (2 * A0 * hessian_mag[0][0] - 2 * B0 * hessian_mag[0][1] \
                                   + beta_I * gradient_mag[0])
            g_temp[i + 1] = 1/2 * beta_I * (-2*hessian_mag[0][1] * A0 - 2 * B0 * hessian_mag[1][1] \
                                       + beta_I * gradient_mag[1])
        g[:, ph] = g_temp
        
        b_factor = 0
        #still need capacitance
        #still need loads
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])   
        Sk = dss.CktElement.Powers()
        if cplx == 'im':
            b_factor = Sk[1]
            
        elif cplx == 're':
            b_factor = Sk[0]
            
        b_temp = beta_S \
            + (1/2*beta_I) * (2 * hessian_mag[0][1] * A0 * B0 + hessian_mag[0][0] * A0**2 + hessian_mag[1][1] * B0**2) \
            - beta_I * (gradient_mag[0] + gradient_mag[1]) \
            + beta_I * (A0**2 + B0**2) ** (1/2) \
            - b_factor
        b[ph] = b_temp

In [77]:
KCL_residuals('cplx')